## Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('train_original.csv')

In [ ]:
train.head(10)

## Keyword

In [ ]:
train['keyword'] = train['keyword'].str.replace('%20', ' ')

## Location

In [ ]:
train['location'] = train['location'].str.replace('United States', 'USA', case=False)
train['location'] = train['location'].replace('US', 'USA')

In [ ]:
train['location'] = train['location'].str.replace('United Kingdom', 'UK', case=False)
train['location'] = train['location'].str.replace('California, USA', 'California', case=False)
train['location'] = train['location'].str.replace('New York, NY', 'New York', case=False)
train['location'] = train['location'].str.replace('New York ,NY', 'New York', case=False)
train['location'] = train['location'].str.replace('New York, USA', 'New York', case=False)
train['location'] = train['location'].str.replace('New York City', 'New York', case=False)
train['location'] = train['location'].str.replace('NYC', 'New York', case=False)
train['location'] = train['location'].str.replace('Washington, D.C.', 'Washington, DC', case=False)
train['location'] = train['location'].str.replace('Los Angeles, CA', 'Los Angeles', case=False)
train['location'] = train['location'].str.replace('London, UK', 'London', case=False)
train['location'] = train['location'].replace('worldwide', 'Worldwide')
train['location'] = train['location'].replace('Denver, Colorado', 'Devnver, CO')
train['location'] = train['location'].replace('Seattle, WA', 'Seattle')
train['location'] = train['location'].str.replace('Earth', 'Worldwide', case=False)
train['location'] = train['location'].str.replace('Everywhere', 'Worldwide', case=False)
train['location'] = train['location'].str.replace('San Francisco, CA', 'San Francisco', case=False)
train['location'] = train['location'].str.replace('London, England', 'London', case=False)
train['location'] = train['location'].str.replace('Chicago, IL', 'Chicago', case=False)
train['location'] = train['location'].replace('World', 'Worldwide')
train['location'] = train['location'].replace('M!$$!$$!PP!', 'Mississippi')

## Text

In [ ]:
english = 'abcdefghijklmnopqrstuvwxyz0123456789 '
text = {}
len_english = len(english)
len_train = len(train['text'])
for i in range(0, len_train):
    len_text = len(train['text'].iloc[i])
    for j in range(0, len_text):
        lower_case = train['text'].iloc[i].lower()
        special_character = []
        if lower_case[j] not in english:
            if lower_case[j] in text:
                special_character = text[lower_case[j]]
            special_character.append(i)
            text[lower_case[j]] = special_character

In [ ]:
text.keys()

In [ ]:
def show_sample():
    for key in text.keys():
        sample = text[key][0]
        print(train['text'].iloc[sample])
        
show_sample()

In [ ]:
import re

In [ ]:
train['text'] = train['text'].str.replace('&amp', 'and')
train['text'] = train['text'].str.replace('amp;', 'and')
train['text'] = train['text'].str.replace('~', ' ')
train['text'] = train['text'].str.replace('Ûª', '\'')
train['text'] = train['text'].str.replace('ÛÒ', ' ')
train['text'] = train['text'].str.replace('ÛÓ', ' ')
train['text'] = train['text'].str.replace('&gt;', ' ')
train['text'] = train['text'].str.replace('&lt;', ' ')
train['text'] = train['text'].str.replace('Û÷', '')
train['text'] = train['text'].str.replace('ÛÏ', ' ')
train['text'] = train['text'].str.replace('Û', ' ')
train['text'] = train['text'].str.replace('\n', ' ')
train['text'] = train['text'].str.replace('å«', ' ')
train['text'] = train['text'].str.replace('åÊ', ' ')
train['text'] = train['text'].str.replace('åÇ', ' ')
train['text'] = train['text'].str.replace('Ì©', 'e')
train['text'] = train['text'].str.replace('Ì¤', 'c')
train['text'] = train['text'].str.replace('Ì¼', 'u')
train['text'] = train['text'].str.replace('Ì_', 'o')
train['text'] = train['text'].str.replace('`', '\'' )
train['text'] = train['text'].str.replace('\x89', ' ')
train['text'] = train['text'].str.replace('\x9d', ' ')

#### Buscamos palabras/signos que hayan quedado

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk #pip install nltk
nltk.download('punkt')
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False, ngram_range=(1,1), stop_words='english', tokenizer=nltk.word_tokenize)
tf_idf_data = tf_idf_vec.fit_transform(train['text'])
col_names = tf_idf_vec.get_feature_names()

In [ ]:
col_names

In [ ]:
len(col_names)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

porter=PorterStemmer()

def clean_text(text):
    token_text = word_tokenize(text.lower())
    new_text = []
    for word in token_text:
        word = re.sub("(http|https)", "", word)
        word = re.sub("//t\.co/(.*)", "link", word)
        word = re.sub("n't", " not", word)
        word = re.sub("\'s", " is", word)
        word = re.sub("\'re", " are", word)
        word = re.sub("\.|/|\+|-|:", " ", word)
        word = re.sub("( ' )|( # )", " ", word)
        word = re.sub(r"åÈ|&;|Û_|å¨|ã¢|ÌÑ|Ì¢|¤}|Ìü|Ì´|:\\\\\\|\\|[\'©¢åâÂ¡^¬=<>\|\[\]\(\)\{\}_,#;¡!¿?£$\*¼%@¤]", "", word)
        word = re.sub(" 0+", " 0 ", word)
        new_text.append(porter.stem(word))
        new_text.append(" ")
    
    return "".join(new_text)

In [ ]:
train['text'] = train['text'].apply(clean_text)

In [ ]:
show_sample()

## Resultado

In [ ]:
train.head(10)

## Guardado del dataframe

In [ ]:
train.to_csv('train.csv', index=False)